# Day 1 - Lab 1: AI-Powered Requirements & User Stories (Solution)

**Objective:** Use a Large Language Model (LLM) to decompose a vague problem statement into structured features, user personas, and Agile user stories, culminating in a machine-readable JSON artifact.

**Introduction:**
This notebook contains the complete solution for Lab 1. It demonstrates how to use an LLM to systematically break down a problem, generate structured requirements, and programmatically validate the output. Each step includes explanations of the code and the reasoning behind the prompts.

## Step 1: Setup

**Purpose:** This initial block of code prepares our environment for the lab. It imports necessary libraries and configures the connection to the LLM API.

**Libraries Explained:**
- **`os`**: A standard Python library for interacting with the operating system. We use `os.getenv` to securely read API keys stored in environment variables, which is a best practice to avoid hardcoding secrets in the code.
- **`json`**: A standard library for working with JSON data. We use `json.loads` to parse the LLM's text output into a Python dictionary or list, and `json.dumps` to format Python objects into a pretty-printed JSON string for saving.
- **`utils`**: A custom helper script provided with this course. 
  - `setup_llm_client()`: This function handles the logic of reading your `.env` file and initializing the correct API client (e.g., for OpenAI). It centralizes the setup code so we don't have to repeat it in every notebook.
  - `get_completion()`: This function takes a prompt and the client information, sends the request to the LLM, and returns the text response. It simplifies the API call process.
  - `save_artifact()`: This function takes content and a file path, creates the necessary directories, and saves the content to a file. It ensures our project artifacts are stored consistently.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    # This works when running as a script
    project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
except NameError:
    # This works when running in an interactive environment (like a notebook)
    # We go up two levels from the notebook's directory to the project root.
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [8]:
import os
import json
from utils import setup_llm_client, get_completion, save_artifact

# This function reads your .env file, finds your API key, and sets up the client.
# It returns the client object and model information for use in other functions.
client, model_name, api_provider = setup_llm_client()

## Step 2: The Problem Statement

We define our starting point—a simple, high-level problem statement—as a Python variable. This makes it easy to reuse in multiple prompts.

In [10]:
problem_statement = input("Define your problem")

Define your problem We need a tool to help our company's new hires get up to speed.


## Step 3: The Challenges

Here are the complete solutions for each challenge.

### Challenge 1 (Foundational): Brainstorming Features

**Explanation:**
This first challenge is about exploration. We use simple, direct prompts to get the LLM's initial thoughts on the problem. The goal is to generate a broad set of ideas (features and personas) that will serve as the raw material for the more structured tasks to follow. We expect the output to be human-readable markdown.

In [11]:
# This prompt is direct and open-ended, encouraging the LLM to be creative.
features_prompt = f"""
Based on the problem statement: '{problem_statement}', brainstorm a list of potential features for a new hire onboarding tool. 
Format the output as a simple markdown list.
"""

print("--- Brainstorming Features ---")
brainstormed_features = get_completion(features_prompt, client, model_name, api_provider)

# This prompt asks for specific roles to ground the brainstorming in user-centric thinking.
personas_prompt = f"""
Based on the problem statement: '{problem_statement}', identify and describe three distinct user personas who would interact with this tool. 
For each persona, describe their role and main goal.
"""

print("\n--- Identifying User Personas ---")
user_personas = get_completion(personas_prompt, client, model_name, api_provider)

--- Brainstorming Features ---


**Error:** An API error occurred: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-svcac************************************************************************************************************************************************************tAA.. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


--- Identifying User Personas ---


**Error:** An API error occurred: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-svcac************************************************************************************************************************************************************tAA.. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### Challenge 2 (Intermediate): Generating Formal User Stories

**Explanation:**
This challenge represents a significant increase in complexity and value. We are no longer asking for simple text; we are demanding a specific, structured data format (JSON). 

The prompt is carefully engineered:
1.  **Persona:** `You are a Senior Product Manager...` tells the LLM the role it should adopt.
2.  **Context:** We provide the previous outputs (`problem_statement`, `brainstormed_features`, `user_personas`) inside `<context>` tags to give the LLM all the necessary information.
3.  **Format:** The `OUTPUT REQUIREMENTS` section is extremely explicit. It tells the LLM to *only* output JSON, defines the exact keys for each object, and specifies the format for nested data (like the array of Gherkin strings). This strictness is key to getting reliable, machine-readable output.
4.  **Parsing:** The `try...except` block is a crucial step. It attempts to parse the LLM's string output into a Python list of dictionaries. If it succeeds, we know the LLM followed our instructions perfectly. If it fails, we print the raw output to help debug the prompt.

In [ ]:
# The prompt is highly-structured to guide the LLM toward a perfect JSON output.
json_user_stories_prompt = f"""
You are a Senior Product Manager creating a product backlog for a new hire onboarding tool.

Based on the following context:
<context>
Problem Statement: {problem_statement}
Potential Features: {brainstormed_features}
User Personas: {user_personas}
</context>

Your task is to generate a list of 5 detailed user stories.

**OUTPUT REQUIREMENTS**:
- You MUST output a valid JSON array. Do not include any text or markdown before or after the JSON array.
- Each object in the array must represent a single user story.
- Each object must have the following keys: 'id' (an integer), 'persona' (a string from the personas), 'user_story' (a string in the format 'As a [persona], I want [goal], so that [benefit].'), and 'acceptance_criteria' (an array of strings, with each string in Gherkin format 'Given/When/Then').
"""

print("--- Generating User Stories as JSON ---")
# We set a lower temperature to encourage the LLM to stick to the requested format.
json_output_str = get_completion(json_user_stories_prompt, client, model_name, api_provider, temperature=0.2)

# Attempt to parse the string output into a Python list.
try:
    # In some cases, the LLM might wrap the JSON in markdown fences (```json ... ```).
    # This line cleans that up before parsing.
    if '```' in json_output_str:
        json_output_str = json_output_str.split('```')[1].lstrip('json').strip()
    
    user_stories_json = json.loads(json_output_str)
    print("Successfully parsed LLM output as JSON.")
    # Pretty-print the first user story to verify its structure
    print("\n--- Sample User Story ---")
    print(json.dumps(user_stories_json[0], indent=2))
except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"Error: Failed to parse LLM output as JSON. Error: {e}")
    print("LLM Output was:\n", json_output_str)
    user_stories_json = [] # Assign an empty list to prevent errors in the next cell

### Challenge 3 (Advanced): Programmatic Validation and Artifact Creation

**Explanation:**
This is the final and most critical step. We treat the LLM's output as untrusted input and subject it to programmatic validation. This ensures that the artifact we create is reliable and conforms to our project's standards. 

The `validate_and_save_stories` function acts as a gatekeeper. It checks for the correct data types (a list of objects) and ensures that all required fields are present in each object. Only if all checks pass do we proceed to save the file using `save_artifact`. This creates a trustworthy `user_stories.json` file that can be confidently used as an input for other automated processes in our SDLC.

In [ ]:
def validate_and_save_stories(stories_data):
    """Validates the structure of the user stories data and saves it if valid."""
    # Ensure the input is a list and not empty.
    if not isinstance(stories_data, list) or not stories_data:
        print("Validation Failed: Data is not a non-empty list.")
        return

    required_keys = ['id', 'persona', 'user_story', 'acceptance_criteria']
    is_valid = True

    # Iterate through each story object in the list.
    for i, story in enumerate(stories_data):
        # Check for the presence of all required keys.
        if not all(key in story for key in required_keys):
            print(f"Validation Failed: Story at index {i} is missing one or more required keys.")
            is_valid = False
            continue # Don't bother with further checks for this invalid story
        
        # Check that the acceptance criteria is a list with at least one item.
        ac = story.get('acceptance_criteria')
        if not isinstance(ac, list) or not ac:
            print(f"Validation Failed: Story at index {i} ('{story.get('id')}') has invalid or empty acceptance criteria.")
            is_valid = False

    # Only save the artifact if all stories in the list are valid.
    if is_valid:
        print("\nAll user stories passed validation.")
        artifact_path = "artifacts/user_stories.json"
        
        # Use the helper function to save the file, creating the 'artifacts' directory if needed.
        # We use json.dumps with an indent to make the saved file human-readable.
        save_artifact(json.dumps(stories_data, indent=2), artifact_path)
    else:
        print("\nValidation failed. Artifact not saved.")

# Run the validation function on the data we parsed from the LLM.
if 'user_stories_json' in locals() and user_stories_json:
    validate_and_save_stories(user_stories_json)
else:
    print("Skipping validation as user_stories_json is empty or could not be parsed.")

## Lab Conclusion

Congratulations! You have completed the first lab. You started with a vague, one-sentence problem and finished with a structured, validated, machine-readable requirements artifact. This is the critical first step in an AI-assisted software development lifecycle. The `user_stories.json` file you created will be the direct input for our next lab, where we will generate a formal Product Requirements Document (PRD).